In [ ]:
import os
import cv2
import numpy as np

def resize_image(image, width=224, height=224):
    return cv2.resize(image, (width, height))

def apply_histogram_equalization(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)
    lab_eq = cv2.merge((l_eq, a, b))
    return cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

def apply_sobel_edge(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
    normalized = np.uint8(255 * magnitude / np.max(magnitude))
    return normalized

# --------------------
# AUGMENTATION FUNCTIONS
# --------------------
def rotate_image(image):
    return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)

def flip_image(image):
    return cv2.flip(image, 1)

def zoom_image(image, zoom_factor=0.8):
    h, w = image.shape[:2]
    zoomed = image[int(h*(1-zoom_factor)/2):int(h*(1+zoom_factor)/2), int(w*(1-zoom_factor)/2):int(w*(1+zoom_factor)/2)]
    return cv2.resize(zoomed, (w, h))

def save_augmented_images(image, base_output_path, base_name):
    augmentations = {
        'rotate': rotate_image(image),
        'flip': flip_image(image),
        'zoom': zoom_image(image)
    }
    for aug_name, aug_img in augmentations.items():
        output_path = os.path.join(base_output_path, f"{base_name}_{aug_name}.png")
        cv2.imwrite(output_path, aug_img)

# --------------------
# PROCESSING FUNCTION
# --------------------
def process_images(input_dir, output_dir):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if not file.lower().endswith('.png'):
                continue
            
            image_path = os.path.join(root, file)
            rel_path = os.path.relpath(root, input_dir)
            base_output_path = os.path.join(output_dir, rel_path)
            os.makedirs(base_output_path, exist_ok=True)

            image = cv2.imread(image_path)
            if image is None:
                print(f"Could not read {image_path}")
                continue

            # Resize
            resized = resize_image(image)

            # Histogram Equalization
            hist_eq = apply_histogram_equalization(resized)
            hist_eq_path = os.path.join(base_output_path, f"{os.path.splitext(file)[0]}_histeq.png")
            cv2.imwrite(hist_eq_path, hist_eq)

            # Sobel Edge Detection
            sobel_edge = apply_sobel_edge(hist_eq)
            sobel_path = os.path.join(base_output_path, f"{os.path.splitext(file)[0]}_sobel.png")
            cv2.imwrite(sobel_path, sobel_edge)

            # Augmentation on Sobel Output
            sobel_color = cv2.cvtColor(sobel_edge, cv2.COLOR_GRAY2BGR)  # Convert back for augmentation
            save_augmented_images(sobel_color, base_output_path, f"{os.path.splitext(file)[0]}_sobel")

# --------------------
# MAIN
# --------------------
def main():
    input_folder = "your_input_folder_path_here"
    output_folder = "your_output_folder_path_here"
    process_images(input_folder, output_folder)

if __name__ == "__main__":
    main()
